In [ ]:
!pip install torch-scatter    
!pip install torch-sparse      
!pip install torch-cluster     
!pip install torch-spline-conv
!pip install -q torch-geometric

In [ ]:
import pandas as pd, numpy as np
from itertools import product
import io, os, json
import tqdm

import time

from sklearn.metrics import mean_squared_error
import torch
import torch.nn.functional as F
import torch_geometric.transforms as T

from torch_geometric.nn import to_hetero
from torch_geometric.nn import HeteroConv, GCNConv, SAGEConv, GATConv, Linear, GraphConv
from torch_geometric.data import Data, HeteroData
from torch_geometric.loader import HGTLoader, NeighborLoader, LinkNeighborLoader

import polars as pl

%matplotlib inline

In [ ]:
df = pl.read_parquet('/kaggle/input/otto-full-optimized-memory-footprint/train.parquet')

In [ ]:
df = df.to_pandas()

In [ ]:
df = df.sample(frac=0.3)

In [ ]:
# session index dict
session = df['session'].unique()
source_idx = {id:idx for idx, id in enumerate(session)}

# aid(article id) index dict
aid = df['aid'].unique()
target_idx = {id:idx for idx, id in enumerate(aid)}

In [ ]:
connected = df[['session', 'aid']]
connected['session'] = connected['session'].map(source_idx)
connected['aid'] = connected['aid'].map(target_idx)

In [ ]:
source = connected['session']
target = connected['aid']
edge_index = torch.tensor((source.values, target.values))

In [ ]:
## Nodes Atrributes
session_num_nodes = df['session'].nunique()
aid_num_nodes = df['aid'].nunique()
session_id = torch.tensor(list(source_idx.values()), dtype=torch.int64)

aid_features = torch.rand((aid_num_nodes, 32)) # Create (random) article features with shape [num_node_aid, dimensions]
aid_id = torch.tensor(list(target_idx.values()), dtype=torch.int64)
#aid_features = torch.nn.Embedding(aid_num_nodes, 32)

## Edges Atrributes
edge_index = edge_index
edge_label = torch.tensor(df['type'].values).type(torch.int64)

In [ ]:
import gc
del df
gc.collect()

In [ ]:
node_types = {
    'session': {
        #'num_nodes': session_num_nodes,
        'node_id': session_id
    },
    'aid': {
        'x': aid_features,
        'node_id': aid_id
    }
}

edge_types = {
    ('session', 'event', 'aid'): {
        'edge_index': edge_index,
        'edge_label': edge_label
    }#,
    #('session', 'cart', 'aid'): {
        
    #},
    #('session', 'buy', 'aid'): {
        
    #}
}

In [ ]:
data = HeteroData({**node_types, **edge_types})

In [ ]:
del source, target, edge_index, edge_label
gc.collect()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# add sesion features for message passing:
data['session'].x = torch.rand(data['session'].num_nodes, 32)

In [ ]:
# Add a reverse ('movie', 'rev_rates', 'user') relation for message passing:
data = T.ToUndirected()(data)

In [ ]:
del data['aid', 'rev_event', 'session'].edge_label  # Remove "reverse" label.

In [ ]:
del data['session'].num_nodes

In [ ]:
data

In [ ]:
# Perform a link-level split into training, validation, and test edges:
train_data, val_data, test_data = T.RandomLinkSplit(
    num_val=0.1,
    num_test=0.1,
    neg_sampling_ratio=0.0,
    edge_types=[('session', 'event', 'aid')],
    rev_edge_types=[('aid', 'rev_event', 'session')],
)(data)

In [ ]:
# Define seed edges:
edge_label_index = train_data['session', 'event', 'aid'].edge_label_index
edge_label = train_data['session', 'event', 'aid'].edge_label

train_loader = LinkNeighborLoader(
    data=train_data,  # TODO
    num_neighbors=[3, 1],  # TODO
    neg_sampling_ratio=0.0,  # TODO
    edge_label_index=(('session', 'event', 'aid'), edge_label_index),
    edge_label=edge_label,
    batch_size=128,
    shuffle=True,
)

In [ ]:
#We have an unbalanced dataset with many labels for rating 3 and 4, and very
# few for 0 and 1. Therefore we use a weighted MSE loss.
weight = torch.bincount(train_data['session', 'aid'].edge_label)
weight = weight.max() / weight

In [ ]:

def weighted_mse_loss(pred, target, weight=None):
    weight = 1. if weight is None else weight[target].to(pred.dtype)
    return (weight * (pred - target.to(pred.dtype)).pow(2)).mean()

In [ ]:
class GNNEncoder(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = SAGEConv((-1, -1), hidden_channels)
        self.conv2 = SAGEConv((-1, -1), out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x


class EdgeDecoder(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.lin1 = Linear(2 * hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, 1)

    def forward(self, z_dict, edge_label_index):
        row, col = edge_label_index
        z = torch.cat([z_dict['session'][row], z_dict['aid'][col]], dim=-1)

        z = self.lin1(z).relu()
        z = self.lin2(z)

        return z.view(-1)


class Model(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        # encoder and decoder
        self.encoder = GNNEncoder(hidden_channels, hidden_channels)
        self.encoder = to_hetero(self.encoder, data.metadata(), aggr='sum')
        self.decoder = EdgeDecoder(hidden_channels)

        # embedding matrices for sessions and aids
        #self.aid_lin = Linear(hidden_channels, hidden_channels)
        #self.session_emb = torch.nn.Embedding(data['session'].num_nodes, hidden_channels)
        #self.aid_emb = torch.nn.Embedding(data['aid'].num_nodes, hidden_channels)

    def forward(self, x_dict, edge_index_dict, edge_label_index):
        #x_dict = {
        #    'session': self.session_emb(data['session'].node_id.to(device)),
        #    'aid': self.aid_lin(data['aid'].x.to(device).float()) + self.aid_emb(data['aid'].node_id.to(device))
        #}
        z_dict = self.encoder(x_dict, edge_index_dict)
        return self.decoder(z_dict, edge_label_index)

    def get_embedding(self, x_dict, edge_index_dict):
        return self.encoder(x_dict, edge_index_dict)

In [ ]:
model = Model(hidden_channels=32).to(device)

In [ ]:
sampled_data = next(iter(train_loader))

with torch.no_grad():
    model.encoder(sampled_data.to(device).x_dict, sampled_data.to(device).edge_index_dict)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)


In [ ]:
def train(train_data=sampled_data):
    model.train()
    optimizer.zero_grad()
    pred = model(train_data.x_dict, train_data.edge_index_dict,
                 train_data[('session', 'event', 'aid')].edge_label_index)
    target = train_data['session', 'aid'].edge_label
    loss = weighted_mse_loss(pred, target, weight.to(device))
    #loss = F.binary_cross_entropy_with_logits(pred, target)

    # Embedding
    emb_dict = model.get_embedding(train_data.x_dict, train_data.edge_index_dict)

    loss.backward()
    optimizer.step()
    return float(loss), pred, model, emb_dict

@torch.no_grad()
def test(data=data):
    model.eval()
    #pred = model(data.x_dict, data.edge_index_dict,
    #             data['session', 'aid'].edge_label_index)
    pred = model(data.edge_index_dict,
                 data['session', 'aid'].edge_label_index)
    pred = pred.clamp(min=0, max=2)
    target = data['session', 'aid'].edge_label.float()
    rmse = F.mse_loss(pred, target).sqrt()
    return float(rmse), pred, target

In [ ]:
%%time
import torch.nn.functional as F

#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: '{device}'")

#model = Model(hidden_channels=32).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(0, 1):
    total_loss = total_examples = 0
    for sampled_data in tqdm.tqdm(train_loader):
        sampled_data = sampled_data.to(device)
        optimizer.zero_grad()
        loss, pred, model, emb_dict = train(train_data=sampled_data)
        #loss, pred, model = train(model=model, train_data=sampled_data)
        # TODO: Move `sampled_data` to the respective `device`
        # TODO: Run `forward` pass of the model
        # TODO: Apply binary cross entropy via
        # `F.binary_cross_entropy_with_logits(pred, ground_truth)`

        #loss.backward()
        #optimizer.step()
        total_loss += float(loss) * pred.numel()
        total_examples += pred.numel()
    print(f"Epoch: {epoch:03d}, Loss: {total_loss / total_examples:.4f}")

In [ ]:
path = '/kaggle/working/model.pt'

In [ ]:
# Save the entire model
torch.save(model, path)
model

In [ ]:

# Load the entire model
model = torch.load(path, map_location=torch.device('cpu'))
model

In [ ]:
def evaluate(path,mode="validation",n_neighbors=20):


    test = pl.read_parquet(path)

    session_types = ['clicks', 'carts', 'orders']

    test_session_AIDs = test.to_pandas().reset_index(drop=True).groupby('session')['aid'].apply(list)

    test_session_types = test.to_pandas().reset_index(drop=True).groupby('session')['type'].apply(list)

    del test
    gc.collect()

    labels = []

    type_weight_multipliers = {0: 1, 1: 6, 2: 3}

    for AIDs, types in zip(test_session_AIDs, test_session_types):
        # AIDs: list of connected aids in each session
        # types: list of connected types in each session
        if len(AIDs) >= 20: # this session have at least 20 aids connected
            # if we have enough aids (over equals 20) we don't need to look for candidates! we just use the old logic
            weights = np.logspace(0.1,1,len(AIDs), base=2, endpoint=True) - 1
            aids_temp = defaultdict(lambda: 0)
            for aid, w, t in zip(AIDs, weights, types): 
                # {aid : weight}
                aids_temp[aid] += w * type_weight_multipliers[t]

            # list of sorted dictionary key by value
            sorted_aids = [k for k, v in sorted(aids_temp.items(), key=lambda item: -item[1])]
            labels.append(sorted_aids[:20]) # only top 20
        else:
            # here we don't have 20 aids to output -- we will use word2vec embeddings to generate candidates!
            AIDs = list(dict.fromkeys(AIDs[::-1]))

            # let's grab the most recent aid
            most_recent_aid = AIDs[0]

            # and look for some neighbors!
            nns = [i for i in index.get_nns_by_item(most_recent_aid, n_neighbors+1)[1:]]


            labels.append((AIDs+nns)[:n_neighbors])

    labels_as_strings = [' '.join([str(l) for l in lls]) for lls in labels]

    predictions = pd.DataFrame(data={'session_type': test_session_AIDs.index, 'labels': labels_as_strings})

    prediction_dfs = []

    for st in session_types:
        modified_predictions = predictions.copy()
        modified_predictions.session_type = modified_predictions.session_type.astype('str') + f'_{st}'
        prediction_dfs.append(modified_predictions)

    sub = pd.concat(prediction_dfs).reset_index(drop=True)
    
    del prediction_dfs, predictions,labels_as_strings, labels, test_session_types,test_session_AIDs
    gc.collect()
    if mode=="test":
        sub.to_csv("submission.csv",index=False)
        return sub
    else:
        sub['labels_2'] = sub['labels'].apply(lambda x : [int(s) for s in x.split(' ')])
        submission = pd.DataFrame()
        submission['session'] = sub.session_type.apply(lambda x: int(x.split('_')[0]))
        submission['type'] = sub.session_type.apply(lambda x: x.split('_')[1])
        submission['labels'] = sub.labels_2.apply(lambda x : [item for item in x[:] ]) #.apply(lambda x: [int(i) for i in x.split(',')[:20]])
        test_labels = pd.read_parquet('/content/drive/MyDrive/OTTO-Kaggle-Competition/for-local-validation/test_labels.parquet')
        test_labels = test_labels.merge(submission, how='left', on=['session', 'type'])
        del sub,submission
        gc.collect()
        gc.collect()
        test_labels['hits'] = test_labels.apply(lambda df: len(set(df.ground_truth).intersection(set(df.labels))), axis=1)
        test_labels['gt_count'] = test_labels.ground_truth.str.len().clip(0,20)
        recall_per_type = test_labels.groupby(['type'])['hits'].sum() / test_labels.groupby(['type'])['gt_count'].sum() 
        score = (recall_per_type * pd.Series({'clicks': 0.1, 'carts': 0.30, 'orders': 0.60})).sum()

        return score

In [ ]:
path = "/kaggle/input/otto-full-optimized-memory-footprint/test.parquet"
test_submission = evaluate(path,mode="test",n_neighbors=20)